# Assignment 2

In this assigment, we will work with the *Forest Fire* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/162/forest+fires). Extract the data files into the subdirectory: `../data/fires/` (relative to `./05_src/`).

## Objective

+ The model objective is to predict the area affected by forest fires given the features set. 
+ The objective of this exercise is to assess your ability to construct and evaluate model pipelines.
+ Please note: the instructions are not meant to be 100% prescriptive, but instead they are a set of minimum requirements. If you find predictive performance gains by applying additional steps, by all means show them. 

## Variable Description

From the description file contained in the archive (`forestfires.names`), we obtain the following variable descriptions:

1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
3. month - month of the year: "jan" to "dec" 
4. day - day of the week: "mon" to "sun"
5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
6. DMC - DMC index from the FWI system: 1.1 to 291.3 
7. DC - DC index from the FWI system: 7.9 to 860.6 
8. ISI - ISI index from the FWI system: 0.0 to 56.10
9. temp - temperature in Celsius degrees: 2.2 to 33.30
10. RH - relative humidity in %: 15.0 to 100
11. wind - wind speed in km/h: 0.40 to 9.40 
12. rain - outside rain in mm/m2 : 0.0 to 6.4 
13. area - the burned area of the forest (in ha): 0.00 to 1090.84 









### Specific Tasks

+ Construct four model pipelines, out of combinations of the following components:

    + Preprocessors:

        - A simple processor that only scales numeric variables and recodes categorical variables.
        - A transformation preprocessor that scales numeric variables and applies a non-linear transformation.
    
    + Regressor:

        - A baseline regressor, which could be a [K-nearest neighbours model]() or a linear model like [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) or [Ridge Regressors](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ridge_regression.html).
        - An advanced regressor of your choice (e.g., Bagging, Boosting, SVR, etc.). TIP: select a tree-based method such that it does not take too long to run SHAP further below. 

+ Evaluate tune and evaluate each of the four model pipelines. 

    - Select a [performance metric](https://scikit-learn.org/stable/modules/linear_model.html) out of the following options: explained variance, max error, root mean squared error (RMSE), mean absolute error (MAE), r-squared.
    - *TIPS*: 
    
        * Out of the suggested metrics above, [some are correlation metrics, but this is a prediction problem](https://www.tmwr.org/performance#performance). Choose wisely (and don't choose the incorrect options.) 

+ Select the best-performing model and explain its predictions.

    - Provide local explanations.
    - Obtain global explanations and recommend a variable selection strategy.

+ Export your model as a pickle file.


You can work on the Jupyter notebook, as this experiment is fairly short (no need to use sacred). 

# Load the data

Place the files in the ../../05_src/data/fires/ directory and load the appropriate file. 

In [270]:
%load_ext dotenv
%dotenv 

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [271]:
# Load the libraries as required.
import os
import sys
sys.path.append(os.getenv('SRC_DIR'))
import numpy as np
import os

In [272]:
import pandas as pd

In [273]:
# Load data
columns = [
    'coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain', 'area' 
]
fires_dt = (pd.read_csv('../../05_src/data/fires/forestfires.csv', header = 0, names = columns))
fires_dt.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   coord_x  517 non-null    int64  
 1   coord_y  517 non-null    int64  
 2   month    517 non-null    object 
 3   day      517 non-null    object 
 4   ffmc     517 non-null    float64
 5   dmc      517 non-null    float64
 6   dc       517 non-null    float64
 7   isi      517 non-null    float64
 8   temp     517 non-null    float64
 9   rh       517 non-null    int64  
 10  wind     517 non-null    float64
 11  rain     517 non-null    float64
 12  area     517 non-null    float64
dtypes: float64(8), int64(3), object(2)
memory usage: 52.6+ KB


In [274]:
fires_dt

,coord_x,coord_y,month,day,ffmc,dmc,dc,isi,temp,rh,wind,rain,area
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.00
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.00
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.00
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.00
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,4,3,aug,sun,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,6.44
513,2,4,aug,sun,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,54.29
514,7,4,aug,sun,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0,11.16
515,1,4,aug,sat,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0,0.00


# Get X and Y

Create the features data frame and target data.

In [275]:
target = fires_dt['area']
target


0       0.00
1       0.00
2       0.00
3       0.00
4       0.00
       ...  
512     6.44
513    54.29
514    11.16
515     0.00
516     0.00
Name: area, Length: 517, dtype: float64

In [276]:
features_df =  fires_dt[['coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain']]
features_df

,coord_x,coord_y,month,day,ffmc,dmc,dc,isi,temp,rh,wind,rain
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
512,4,3,aug,sun,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0
513,2,4,aug,sun,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0
514,7,4,aug,sun,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0
515,1,4,aug,sat,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0


# Preprocessing

Create two [Column Transformers](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), called preproc1 and preproc2, with the following guidelines:

- Numerical variables

    * (Preproc 1 and 2) Scaling: use a scaling method of your choice (Standard, Robust, Min-Max). 
    * Preproc 2 only: 
        
        + Choose a transformation for any of your input variables (or several of them). Evaluate if this transformation is convenient.
        + The choice of scaler is up to you.

- Categorical variables: 
    
    * (Preproc 1 and 2) Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) where appropriate.


+ The only difference between preproc1 and preproc2 is the non-linear transformation of the numerical variables.
    


In [277]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import GaussianNB

### Preproc 1

Create preproc1 below.

+ Numeric: scaled variables, no other transforms.
+ Categorical: one-hot encoding.

In [278]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
std_scaler = StandardScaler()
oh_encoder = OneHotEncoder()


In [279]:
prepro_1 = ColumnTransformer(
    transformers=[('num_scaler', StandardScaler(), features_df.drop(['month','day'],axis=1).columns),
                     ('oh_encoder', OneHotEncoder(handle_unknown='infrequent_if_exist'), ['month','day'])
                     ], remainder='drop'
)

prepro_1

ColumnTransformer(transformers=[('num_scaler', StandardScaler(),
                                 Index(['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind',
       'rain'],
      dtype='object')),
                                ('oh_encoder',
                                 OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                 ['month', 'day'])])

In [280]:
# str_vals = features_df[['month', 'day']]
# std_scaler.fit(features_df.drop(str_vals, axis=1))

In [281]:
# onehot = OneHotEncoder()
# onehot.fit(str_vals)

In [282]:
# trans_str = onehot.transform(str_vals)
# # trans_str.toarray()
# trans_str.shape

### Preproc 2

Create preproc1 below.

+ Numeric: scaled variables, non-linear transformation to one or more variables.
+ Categorical: one-hot encoding.

In [283]:
from sklearn.preprocessing import PowerTransformer
pt_scaler = PowerTransformer()

In [284]:
prepro_2 = ColumnTransformer(
    transformers=[('non_lin_trans', PowerTransformer(method = 'yeo-johnson'), features_df.drop(['month','day'],axis=1).columns),
                     ('oh_encoder', OneHotEncoder(handle_unknown='infrequent_if_exist'), ['month','day'])
                     ], remainder='drop'
)

prepro_2


ColumnTransformer(transformers=[('non_lin_trans', PowerTransformer(),
                                 Index(['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind',
       'rain'],
      dtype='object')),
                                ('oh_encoder',
                                 OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                 ['month', 'day'])])

## Model Pipeline


Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `regressor` and assign a regression model to it. 

## Regressor

+ Use a regression model to perform a prediction. 

    - Choose a baseline regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Choose a more advance regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Both model choices are up to you, feel free to experiment.

In [285]:
# Pipeline A = preproc1 + baseline
preprocessing = prepro_1
regressor = LinearRegression()

pipe_simple_1 = Pipeline([
    ('preprocess', preprocessing),
    ('model', regressor)
]) 

pipe_simple_1

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num_scaler',
                                                  StandardScaler(),
                                                  Index(['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind',
       'rain'],
      dtype='object')),
                                                 ('oh_encoder',
                                                  OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                                  ['month', 'day'])])),
                ('model', LinearRegression())])

In [286]:
pipe_simple_1.get_params()

{'memory': None,
 'steps': [('preprocess',
   ColumnTransformer(transformers=[('num_scaler', StandardScaler(),
                                    Index(['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind',
          'rain'],
         dtype='object')),
                                   ('oh_encoder',
                                    OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                    ['month', 'day'])])),
  ('model', LinearRegression())],
 'verbose': False,
 'preprocess': ColumnTransformer(transformers=[('num_scaler', StandardScaler(),
                                  Index(['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind',
        'rain'],
       dtype='object')),
                                 ('oh_encoder',
                                  OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                  ['month', 'day'])]),
 'model': LinearRegression(),
 'preprocess__n_jobs': 

In [287]:
# Pipeline B = preproc2 + baseline

preprocessing = prepro_2
regressor = LinearRegression()

pipe_simple_2 = Pipeline([
    ('preprocess', preprocessing),
    ('model', regressor)
]) 

pipe_simple_2

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('non_lin_trans',
                                                  PowerTransformer(),
                                                  Index(['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind',
       'rain'],
      dtype='object')),
                                                 ('oh_encoder',
                                                  OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                                  ['month', 'day'])])),
                ('model', LinearRegression())])

In [288]:
pipe_simple_2.get_params()

{'memory': None,
 'steps': [('preprocess',
   ColumnTransformer(transformers=[('non_lin_trans', PowerTransformer(),
                                    Index(['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind',
          'rain'],
         dtype='object')),
                                   ('oh_encoder',
                                    OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                    ['month', 'day'])])),
  ('model', LinearRegression())],
 'verbose': False,
 'preprocess': ColumnTransformer(transformers=[('non_lin_trans', PowerTransformer(),
                                  Index(['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind',
        'rain'],
       dtype='object')),
                                 ('oh_encoder',
                                  OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                  ['month', 'day'])]),
 'model': LinearRegression(),
 'preprocess_

In [289]:
from sklearn.tree import DecisionTreeRegressor

In [290]:
# Pipeline C = preproc1 + advanced model

preprocessing = prepro_1
regressor = DecisionTreeRegressor()

pipe_simple_3 = Pipeline([
    ('preprocess', preprocessing),
    ('model', regressor)
]) 

pipe_simple_3


Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num_scaler',
                                                  StandardScaler(),
                                                  Index(['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind',
       'rain'],
      dtype='object')),
                                                 ('oh_encoder',
                                                  OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                                  ['month', 'day'])])),
                ('model', DecisionTreeRegressor())])

In [291]:
pipe_simple_3.get_params()

{'memory': None,
 'steps': [('preprocess',
   ColumnTransformer(transformers=[('num_scaler', StandardScaler(),
                                    Index(['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind',
          'rain'],
         dtype='object')),
                                   ('oh_encoder',
                                    OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                    ['month', 'day'])])),
  ('model', DecisionTreeRegressor())],
 'verbose': False,
 'preprocess': ColumnTransformer(transformers=[('num_scaler', StandardScaler(),
                                  Index(['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind',
        'rain'],
       dtype='object')),
                                 ('oh_encoder',
                                  OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                  ['month', 'day'])]),
 'model': DecisionTreeRegressor(),
 'preprocess_

In [292]:
# Pipeline D = preproc2 + advanced model

preprocessing = prepro_2
regressor = DecisionTreeRegressor()

pipe_simple_4 = Pipeline([
    ('preprocess', preprocessing),
    ('model', regressor)
]) 

pipe_simple_4

    

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('non_lin_trans',
                                                  PowerTransformer(),
                                                  Index(['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind',
       'rain'],
      dtype='object')),
                                                 ('oh_encoder',
                                                  OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                                  ['month', 'day'])])),
                ('model', DecisionTreeRegressor())])

In [293]:
pipe_simple_4.get_params()

{'memory': None,
 'steps': [('preprocess',
   ColumnTransformer(transformers=[('non_lin_trans', PowerTransformer(),
                                    Index(['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind',
          'rain'],
         dtype='object')),
                                   ('oh_encoder',
                                    OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                    ['month', 'day'])])),
  ('model', DecisionTreeRegressor())],
 'verbose': False,
 'preprocess': ColumnTransformer(transformers=[('non_lin_trans', PowerTransformer(),
                                  Index(['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind',
        'rain'],
       dtype='object')),
                                 ('oh_encoder',
                                  OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                  ['month', 'day'])]),
 'model': DecisionTreeRegressor(),
 'p

# Tune Hyperparams

+ Perform GridSearch on each of the four pipelines. 
+ Tune at least one hyperparameter per pipeline.
+ Experiment with at least four value combinations per pipeline.

In [294]:
from sklearn.metrics import mean_squared_error, r2_score

In [322]:
# simple train-test split

X = features_df
Y = fires_dt['area']

scoring = 'neg_root_mean_squared_error'

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)



In [323]:
param_grid = {
     'model__fit_intercept': [True, False],
}

In [297]:
grid_cv_1 = GridSearchCV(
    estimator = pipe_simple_1,
    param_grid = param_grid,
    scoring = scoring,
    cv = 5,
)
grid_cv_1.fit(X_train, Y_train)


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocess',
                                        ColumnTransformer(transformers=[('num_scaler',
                                                                         StandardScaler(),
                                                                         Index(['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind',
       'rain'],
      dtype='object')),
                                                                        ('oh_encoder',
                                                                         OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                                                         ['month',
                                                                          'day'])])),
                                       ('model', LinearRegression())]),
             param_grid={'model__fit_intercept': [True, False]},
             scoring='neg_root_mean_squared_error')

In [298]:

res = grid_cv_1.cv_results_
res = pd.DataFrame(res)
res.columns

res[['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
       'param_model__fit_intercept', 'params', 'split0_test_score',
       'split1_test_score', 'split2_test_score', 'split3_test_score',
       'split4_test_score', 'mean_test_score', 'std_test_score',
       'rank_test_score']].sort_values('rank_test_score')


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_model__fit_intercept,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
1,0.024162,0.007069,0.012189,0.003123,False,{'model__fit_intercept': False},-40.776734,-19.578114,-33.367526,-85.264210,-26.878417,-41.173000,23.133488,1
0,0.027185,0.002313,0.017991,0.006688,True,{'model__fit_intercept': True},-40.776734,-19.578114,-33.367526,-85.269179,-26.895228,-41.177356,23.133306,2


In [299]:
grid_cv_1.best_params_

{'model__fit_intercept': False}

In [300]:
grid_cv_1.best_estimator_

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num_scaler',
                                                  StandardScaler(),
                                                  Index(['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind',
       'rain'],
      dtype='object')),
                                                 ('oh_encoder',
                                                  OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                                  ['month', 'day'])])),
                ('model', LinearRegression(fit_intercept=False))])

In [324]:
# model!
grid_cv_2 = GridSearchCV(
    estimator = pipe_simple_2,
    param_grid = param_grid, 
    scoring = scoring,
    cv = 5,
)
grid_cv_2.fit(X_train, Y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocess',
                                        ColumnTransformer(transformers=[('non_lin_trans',
                                                                         PowerTransformer(),
                                                                         Index(['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind',
       'rain'],
      dtype='object')),
                                                                        ('oh_encoder',
                                                                         OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                                                         ['month',
                                                                          'day'])])),
                                       ('model', LinearRegression())]),
             param_grid={'model__fit_intercept': [True, False]},
             scoring='neg_root_mean_squared_error')

In [302]:

res = grid_cv_2.cv_results_
res = pd.DataFrame(res)
res.columns

res[['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
       'param_model__fit_intercept', 'params', 'split0_test_score',
       'split1_test_score', 'split2_test_score', 'split3_test_score',
       'split4_test_score', 'mean_test_score', 'std_test_score',
       'rank_test_score']].sort_values('rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_model__fit_intercept,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
1,0.082154,0.025656,0.011595,0.002872,False,{'model__fit_intercept': False},-40.564425,-19.744698,-29.25212,-85.094240,-26.816012,-40.294299,23.379426,1
0,0.075956,0.021002,0.017590,0.012730,True,{'model__fit_intercept': True},-40.564425,-19.744698,-29.25212,-85.094744,-26.833634,-40.297924,23.377588,2


In [303]:
grid_cv_2.best_params_

{'model__fit_intercept': False}

In [318]:
# this is what is geetting saved to the pickle!
grid_cv_2.best_estimator_

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('non_lin_trans',
                                                  PowerTransformer(),
                                                  Index(['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind',
       'rain'],
      dtype='object')),
                                                 ('oh_encoder',
                                                  OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                                  ['month', 'day'])])),
                ('model', LinearRegression(fit_intercept=False))])

In [305]:
param_grid = {
     'preprocess__num_scaler__with_mean': [True, False]
}

In [306]:
grid_cv_3 = GridSearchCV(
    estimator = pipe_simple_3,
    param_grid = param_grid, 
    scoring = scoring,
    cv = 5,
)
grid_cv_3.fit(X_train, Y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocess',
                                        ColumnTransformer(transformers=[('num_scaler',
                                                                         StandardScaler(),
                                                                         Index(['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind',
       'rain'],
      dtype='object')),
                                                                        ('oh_encoder',
                                                                         OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                                                         ['month',
                                                                          'day'])])),
                                       ('model', DecisionTreeRegressor())]),
             param_grid={'preprocess__num_scaler__with_mean': [True, False]},
             scoring='neg_root_mean_squared_error')

In [307]:

res = grid_cv_3.cv_results_
res = pd.DataFrame(res)
res.columns

res[['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
       'param_preprocess__num_scaler__with_mean', 'params',
       'split0_test_score', 'split1_test_score', 'split2_test_score',
       'split3_test_score', 'split4_test_score', 'mean_test_score',
       'std_test_score', 'rank_test_score']].sort_values('rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_preprocess__num_scaler__with_mean,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.027374,0.011326,0.012195,0.005633,True,{'preprocess__num_scaler__with_mean': True},-92.075956,-90.244382,-66.236120,-87.674880,-26.411030,-72.528474,24.866779,1
1,0.016589,0.001020,0.006999,0.001412,False,{'preprocess__num_scaler__with_mean': False},-91.926506,-120.569696,-42.975671,-93.240119,-29.621095,-75.666618,33.995254,2


In [308]:
grid_cv_3.best_params_

{'preprocess__num_scaler__with_mean': True}

In [309]:
grid_cv_3.best_estimator_

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num_scaler',
                                                  StandardScaler(),
                                                  Index(['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind',
       'rain'],
      dtype='object')),
                                                 ('oh_encoder',
                                                  OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                                  ['month', 'day'])])),
                ('model', DecisionTreeRegressor())])

In [312]:
param_grid = {
   'preprocess__non_lin_trans__standardize': [True, False]
}

In [313]:
grid_cv_4 = GridSearchCV(
    estimator = pipe_simple_4,
    param_grid = param_grid,
    scoring = scoring,
    cv = 5,
)
grid_cv_4.fit(X_train, Y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocess',
                                        ColumnTransformer(transformers=[('non_lin_trans',
                                                                         PowerTransformer(),
                                                                         Index(['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind',
       'rain'],
      dtype='object')),
                                                                        ('oh_encoder',
                                                                         OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                                                         ['month',
                                                                          'day'])])),
                                       ('model', DecisionTreeRegressor())]),
             param_grid={'preprocess__non_lin_trans__standardize': [True,
                                                                    False]},
             scoring='neg_root_mean_squared_error')

In [314]:

res = grid_cv_4.cv_results_
res = pd.DataFrame(res)
res.columns

res[['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
       'param_preprocess__non_lin_trans__standardize', 'params',
       'split0_test_score', 'split1_test_score', 'split2_test_score',
       'split3_test_score', 'split4_test_score', 'mean_test_score',
       'std_test_score', 'rank_test_score']].sort_values('rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_preprocess__non_lin_trans__standardize,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.065162,0.011883,0.010594,0.003136,True,{'preprocess__non_lin_trans__standardize': True},-91.624185,-121.09099,-66.340634,-91.388125,-25.721408,-79.233068,31.884427,1
1,0.044174,0.003912,0.007998,0.000893,False,{'preprocess__non_lin_trans__standardize': False},-92.314051,-120.47637,-66.012111,-91.890779,-30.470046,-80.232672,30.265392,2


In [315]:
grid_cv_4.best_params_

{'preprocess__non_lin_trans__standardize': True}

In [316]:
grid_cv_4.best_estimator_

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('non_lin_trans',
                                                  PowerTransformer(),
                                                  Index(['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind',
       'rain'],
      dtype='object')),
                                                 ('oh_encoder',
                                                  OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                                  ['month', 'day'])])),
                ('model', DecisionTreeRegressor())])

Old Cross Validation work - saving the code for future testing:

In [25]:
# res_simple_dict = cross_validate(pipe_simple_1, X_train, Y_train, cv = 5, scoring = scoring)
# res_simple = pd.DataFrame(res_simple_dict).assign(experiment = 1) 
# res_simple

In [26]:
# res_simple_dict = cross_validate(pipe_simple_2, X_train, Y_train, cv = 5, scoring = scoring)
# res_simple = pd.DataFrame(res_simple_dict).assign(experiment = 2) 
# res_simple

In [27]:
# res_simple_dict = cross_validate(pipe_simple_3, X_train, Y_train, cv = 5, scoring = scoring)
# res_simple = pd.DataFrame(res_simple_dict).assign(experiment = 3) 
# res_simple

In [28]:
# res_simple_dict = cross_validate(pipe_simple_4, X_train, Y_train, cv = 5, scoring = scoring)
# res_simple = pd.DataFrame(res_simple_dict).assign(experiment = 4) 
# res_simple

# Evaluate

+ Which model has the best performance?

Pipeline 2 is the best performing model, followed closely by Pipeline 1.

# Export

+ Save the best performing model to a pickle file.

In [317]:
import joblib
joblib.dump(grid_cv_2.best_estimator_, 'best_model.pkl')

['best_model.pkl']

# Explain

+ Use SHAP values to explain the following only for the best-performing model:

    - Select an observation in your test set and explain which are the most important features that explain that observation's specific prediction.

    - In general, across the complete training set, which features are the most and least important.

+ If you were to remove features from the model, which ones would you remove? Why? How would you test that these features are actually enhancing model performance?

In [319]:
import shap

c:\Users\Andrea\miniconda3\envs\dsi_participant\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [337]:
# import shap

# # Get model and preprocessing steps
# model = grid_cv_2.best_estimator_.named_steps['regressor']
# preprocessor = grid_cv_2.best_estimator_.named_steps['preprocess']

# # Transform test data
# X_transformed = preprocessor.transform(X_test)

# feature_names = preprocessor.get_feature_names_out()

# # Create SHAP explainer
# explainer = shap.Explainer(model.predict, X_transformed, feature_names=feature_names)

# # Get SHAP values
# shap_values = explainer(X_transformed)

In [338]:
data_transform = grid_cv_2.best_estimator_.named_steps['preprocess'].transform(X_test)

explainer = shap.explainers.Linear(
    grid_cv_2.best_estimator_.named_steps['model'], 
    data_transform,
    feature_names = grid_cv_2.best_estimator_.named_steps['preprocess'].get_feature_names_out())

shap_values = explainer(data_transform)

In the future `np.bool` will be defined as the corresponding NumPy scalar.


AttributeError: module 'numpy' has no attribute 'bool'.
`np.bool` was a deprecated alias for the builtin `bool`. To avoid this error in existing code, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

*(Answer here.)*

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at the `help` channel. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Cortez,Paulo and Morais,Anbal. (2008). Forest Fires. UCI Machine Learning Repository. https://doi.org/10.24432/C5D88D.